# World Happiness Index

En este proyecto se hace un análisis de la infromación recopilada en la base de datos del Índice Mundial de la Felicidad, en donde se despliega el puntaje de felicidad de 156 países en todo el mundo. A través de distintas variables establecidas en el reporte, se hará un análisis de cómo estas variables influyen en la felicidad percibida y cuál es la variable más significativa.

# 1.- Se importan y descargan las librerías necesarias.

In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from statsmodels.formula.api import ols
from sklearn import linear_model

In [ ]:
mpl.style.use(['seaborn-whitegrid'])

Se establece el estilo de gráficas que usaremos

# 2.- Se importa la base de datos de nuestro Dataset

In [ ]:
df = pd.read_excel('datasets/Happiness Index.xlsx',
                  engine = 'openpyxl')
df.head(2)

La variable dependiente que se va a estudiar es la de 'Score', pues es el resultado final que deseamos estudiar. Las demás columnas son las variables que se pretende que expliquen y justifiquen el 'Score'.

# 3.- Se hace un histograma para observar la acumulación de datos generales.

In [ ]:
hist_score = sns.distplot(df.Score)
plt.title('"Normality" of worldwide happiness level')
plt.xlabel('Happiness score')

Se observa que, de todo el conjunto de datos, la mayoría de los puntajes de felicidad están entre el 4 y el 7, demostrando un ligero sesgo a la derecha.

# 4.- Número y tipo de variables
Ahora que sabemos nuestra variable dependiente, es momento de verificar cuáles y de qué tipo son las variables de nuestra base de datos.

In [ ]:
df.info()

Una vez tengamos esta información, es posible verificar el tipo de cada variable. Todas las variables numéricas son de tipo Float, pues tienen valores decimales. La única variable entera es la del 'Overall Rank', la cual únicamente funciona como un tipo de índice. Existen 2 variables categóricas que nos pueden ayudar a segmentar la información y estudiarla por partes, como los continentes o los países.

# 5.- ¿Qué variable, por sí sola, tiene más correlación con la variable objetivo?

In [ ]:
df['Country or region'] = df['Country or region'].astype('category').cat.codes
df['Continent'] = df['Continent'].astype('category').cat.codes

Como tenemos variables de tipo String, no se les puede insertar al cálculo del coeficiente de correlación, por lo que se les transforma en variables categóricas para que puedan ser consideradas en el cálculo.

In [ ]:
var = ['Score','Country or region', 'Continent',
       'GDP per capita', 'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']
correlation = df[var].corr()
correlation

In [ ]:
sns.heatmap(correlation)

Como se observa tanto en la tabla como en el mapa de calor, las variables con más correlación hacia el 'Score' de felicidad son las siguientes (de mayor a menor):
GDP per capita, Healthy Life Expectancy, Social Support, Continent, Freedom to make life choices, Perceptions of corruption, Generosity, Country.

Debido a que los índices de correlación superiores a 0.6 son considerados como fuertes, las variables a tomar en cuenta de aquí en adelante son las que sean superiores a ese valor, como lo son: GDP per capita, Healthy Life Expectancy, Social Support, Continent

En consecuencia el modelo de regresión será:
Score = b0 + (b1 x GDP per capita) + (b2 x Healthy Life Expectancy) + (b3 x Social Support) + (b4 x Continent)

# 6.- ¿Cómo es la correlación entre las variables y 'Score'?

Se realiza un diagrama de dispersión de cada variable vs Score para observar el comportamiento de la variable dependiente conforme al cambio de la variable independiente.

In [ ]:
sns.lmplot(x = 'GDP per capita', y = 'Score', data = df, line_kws={'color': 'blue'})
plt.title('GDP per capita vs Score')

sns.lmplot(x = 'Healthy life expectancy', y = 'Score', data = df, line_kws={'color': 'red'})
plt.title('Health Life Expectancy vs Score')

sns.lmplot(x = 'Social support', y = 'Score', data = df, line_kws={'color': 'green'})
plt.title('Social Support vs Score')

sns.lmplot(x = 'Continent', y = 'Score', data = df, line_kws={'color': 'purple'})
plt.title('Continent vs Score')

Como se puede observar en las 4 gráficas, el puntaje 'Score' depende en buena medida de todas las variables mencionadas. Existe una correlación fuerte positiva entre las variables independientes con la dependiente, por lo que sí se ve afectado el 'Score' dependiendo del valor de la variable independiente.

# 7.- ¿Existen valores atípicos?

Se hará un Boxplot para cada variable y determinar si existen valores atípicos que puedan sesgar de manera incorrecta nuestros datos.

In [ ]:
sns.boxplot(y='Score', x='Continent', 
                 data=df, 
                 palette="colorblind")

In [ ]:
sns.boxplot(y=df['GDP per capita'])

In [ ]:
sns.boxplot(y=df['Healthy life expectancy'])

In [ ]:
sns.boxplot(y=df['Social support'])

Se observaron muy pocos puntos atípicos en las 4 gráficas que se realizaron. Debido a su corta contidad, se pueden considerar como anomalías y desechar de la base de datos.

# 8.- ¿Cuáles son los resultados del modelo de regresión?

In [ ]:
GDP = df['GDP per capita']
HLE = df['Healthy life expectancy']
SS = df['Social support']
score = df['Score']
cont = df['Continent']

model = ols('Score ~ GDP + HLE + SS + Continent', data=df).fit()
model.summary()

Después de obtener un modelo de regresión completo, nos arrojó un R-cuadrado y un R-cuadrado ajustado de 0.742 y 0.735. Estos coeficientes de correlacion significan un alto nivel de correlación, lo que quiere decir que nuestro modelo tiene capacidad predictiva. Esto nos dice que el modelo es capaz de predecir aproximadamente un 73% de la variabilidad en nuestra variable de respuesta. Además, valores cercanos de R-cuadrado y R-cuadrado ajustado significan certidumbre en el resultado de éstos.

Además, todas nuestras variables tienen un valor P inferior al 0.05 (alfa), lo que nos indica que todas son variables significativas. El valor de Durbin-Watson también nos indica que tenemos un modelo aceptable.

Teniendo esto en cuenta, nuestro modelo de regresión es el siguiente:

Score = 2.3874 + (0.8691 x GDP per capita) + (0.8522 * Healthy life expectancy) + (1.1604 x Social support) + (0.1226 x Continent)



# 9.- Gráfica de dispersión a partir de modelo de regresión

In [ ]:
lm = linear_model.LinearRegression()

X = df[['GDP per capita', 'Healthy life expectancy', 'Social support', 'Continent']]
Y = df['Score']

regr = linear_model.LinearRegression()
regr.fit(X, Y)



In [ ]:
model.coef_